In [1]:
# https://stackoverflow.com/questions/37159070/multiple-linear-regression-model-by-using-tensorflow
# https://donaldpinckney.com/books/pytorch/book/ch2-linreg/2018-03-21-multi-variable.html
# https://www.youtube.com/watch?v=Q4GNLhRtZNc
# https://atmamani.github.io/projects/ml/coursera-gd-multivariate-linear-regression/
# https://online.stat.psu.edu/stat462/sites/onlinecourses.science.psu.edu.stat462/files/05mlr/eq_matrix_notation/index.gif

# Multivariable Logistic Regression for matricies.
# target = flux1 + flux2 +... flux500 + b
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior() 
import numpy as np
import pandas as pd
from numpy import exp

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
#normalize X values to help model converge
def normalize(data):
    mean = np.mean(data, axis=0)
    std = np.std(data, axis=0)
    data = (data - mean) / std
    return data

In [3]:
#retrieve and format data - into labels and examples from the dataset
def features_and_labels(filename_a, filename_b):
    with open(filename_b, 'rb') as f:
        labels = np.load(f).transpose()
    
    data = pd.read_pickle(filename_a)
    #https://stackoverflow.com/questions/22798934/pandas-long-to-wide-reshape-by-two-variables
    data_x = data.pivot(index='id', columns='time', values='FLUX')
    data_x = normalize(data_x)
    data_x = data_x.transpose()
    data_x = data_x.to_numpy()
    return data_x, labels

In [4]:
#Logistic Layer using a sigmoid function
def logistic_layer(y):
    y = np.array(y)
    y = 1 / (1 + exp(-y)) # sigmoid function
    y = y.ravel()
    return y

In [5]:
#Calculate an accuracy metric
def accuracy(predicted_y, true_y):
    true_y = np.array(true_y).ravel()
    counter = 0
    for i in range(len(true_y)):
        p_y = predicted_y[i]
        t_y = true_y[i]
        if (p_y>.5 and t_y == 1) or (p_y < .5 and t_y == 0):
            counter+=1
    counter = (counter/ len(true_y)) * 100
    return counter

In [6]:
training_data_x, training_data_y = features_and_labels("../Kepler-Train.pkl", "../Labels-Train.npy")
print(training_data_x.shape, training_data_y.shape)
#set hyperparameters & variables
learning_rate = 0.003
epochs = 50
display_step = 5
n_samples = training_data_x.shape[1]
col_num = training_data_x.shape[0]

X = tf.placeholder(tf.float32, [col_num, n_samples])
Y = tf.placeholder(tf.float32, [1, n_samples]) #resulting dimenstion of W*X matmul

(201, 12589) (1, 12589)


In [7]:
# We want the weight vector to correspond one to one with every column
W = tf.Variable(tf.zeros([1,col_num], dtype=np.float32), name="weight")
b = tf.Variable(tf.zeros([1, ], dtype=np.float32), name="bias")

#matrix multiplication requires outer dimension of W to be equal to be equal to the inner dimension of X: 
# (1,col_num) & (col_num, num_samples) - this is why we transpose X
pred = tf.matmul(W, X) + b # y′(x,A,b)=Ax+b linear matrix equation

error = tf.reduce_sum((pred-Y)**2) / (n_samples * 2) #MSE

In [8]:
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(error)

session = tf.Session()
session.run(tf.global_variables_initializer())

loss_arr = []
acc_arr = []

for t in range(epochs):
    
    _, current_loss, current_W, current_b = session.run([optimizer, error, W, b], feed_dict={
        X: training_data_x,
        Y: training_data_y
    })
    
    loss_arr.append(current_loss)
    acc_arr.append(accuracy(logistic_layer(np.dot(current_W,training_data_x) + current_b), training_data_y))
    
print("Optimization Finished!")

training_error = session.run(error, feed_dict={X: training_data_x, Y: training_data_y})
print("Training error=", training_error, "Weights=", session.run(W), "Bias=", session.run(b), '\n')

2021-12-15 07:51:37.141000: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-15 07:51:37.148393: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-15 07:51:37.148724: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-15 07:51:37.149644: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

Optimization Finished!
Training error= 0.086543135 Weights= [[ 3.7879896e-04  1.8589903e-04  2.8455781e-04  1.7257247e-04
   1.4180588e-05 -1.6046110e-04 -4.5067412e-04 -4.3503122e-04
  -5.1769969e-04 -6.2773179e-04 -7.9175056e-04 -1.0526601e-03
  -1.0928960e-03 -1.1384054e-03 -1.0729275e-03 -1.1507128e-03
  -1.2510265e-03 -1.2776119e-03 -1.2895445e-03 -1.2053946e-03
  -1.1490255e-03 -1.1921276e-03 -1.2573759e-03 -1.0964414e-03
  -1.2391945e-03 -1.4589423e-03 -1.5019424e-03 -1.4921605e-03
  -1.2903677e-03 -9.9651492e-04 -7.9815334e-04 -8.3247176e-04
  -8.1751047e-04 -7.9680525e-04 -6.6964567e-04 -4.7571145e-04
  -4.6622436e-04 -5.4712920e-04 -6.1554811e-04 -6.4099679e-04
  -7.3306699e-04 -6.4235152e-04 -8.2443649e-04 -7.0345268e-04
  -5.3181069e-04 -5.9546588e-04 -7.7271002e-04 -9.5215003e-04
  -1.0682377e-03 -1.3501599e-03 -1.4542186e-03 -1.6000009e-03
  -1.8780994e-03 -2.0361675e-03 -1.8683107e-03 -1.9301062e-03
  -2.0855537e-03 -2.2938803e-03 -2.5902796e-03 -2.1512210e-03
  -2.72021

In [9]:
test_x, test_y = features_and_labels("../Kepler-Test.pkl", "../Labels-Test.npy")

predicted_y = np.dot(session.run(W), test_x) + session.run(b)
predicted_y = logistic_layer(predicted_y)

print("Accuracy percentage: ", accuracy(predicted_y, test_y), "%")

Accuracy percentage:  59.40279542566709 %


In [10]:
with open('tf-models/multi-lr.npy', 'wb') as f:
    np.save(f, session.run(W))
    np.save(f, session.run(b))
    np.save(f, loss_arr)
    np.save(f, acc_arr)
